<a href="https://colab.research.google.com/github/mrhamedani/Gen-AI-projects-Pytorch/blob/main/9_TextGenerationLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 # exercise 1
text="Hi, there!"
tokens=list(text)
print(tokens)

['H', 'i', ',', ' ', 't', 'h', 'e', 'r', 'e', '!']


In [5]:
# exercise 2: word tokenization
text="It is unbelievably good!"
text=text.replace("!","")
tokens=text.split(" ")
print(tokens)

['It', 'is', 'unbelievably', 'good']


In [3]:
# exercise 3 :
text="Hi, there!"
for x in list(",!"):
    text=text.replace(f"{x}",f" {x}")
tokens=text.split(" ")
print(tokens)

['Hi', ',', 'there', '!']


## Download & clean up the text

In [ ]:
!wget -O anna.txt "https://gitlab.com/amirbig44/pytorch-gan-course/-/raw/main/anna.txt?ref_type=heads&inline=false" # text of anna karenina

In [9]:
with open("./anna.txt","r") as f:
    text=f.read()
words=text.split(" ")
print(words[:20])

print(set(text.lower())) # unique characters in the text by set

['Chapter', '1\n\n\nHappy', 'families', 'are', 'all', 'alike;', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own\nway.\n\nEverything', 'was', 'in', 'confusion', 'in', 'the', "Oblonskys'"]
{'.', '7', 'a', 'x', 'o', ')', 'i', '-', '\n', 'e', 'm', 'l', '3', 'b', '5', '?', 'y', "'", 't', ';', 'c', '2', 'j', '6', 'z', 'h', '9', '1', '"', '_', ' ', '!', 'u', 'k', 'r', 'f', ',', '`', 'q', 'w', 'g', '(', ':', 'p', 'd', '0', '8', 'v', '4', 'n', 's'}


In [10]:
clean_text=text.lower().replace("\n", " ")
clean_text=clean_text.replace("-", " ")
for x in ",.:;?!$()/_&%*@'`":
    clean_text=clean_text.replace(f"{x}", f" {x} ")
clean_text=clean_text.replace('"', ' " ')
text=clean_text.split()

In [11]:
from collections import Counter
word_counts = Counter(text)

words=sorted(word_counts, key=word_counts.get, reverse=True) # get unique words
print(words[:10])

[',', '.', 'the', '"', 'and', 'to', 'of', 'he', "'", 'a']


## Here we will make two dictionaries where the key values ​​are in the same place We will use it further

In [13]:
text_length =len(text)
num_unique_words =len(words)
print(f"the text contains {text_length} words")
print(f"there are {num_unique_words} unique tokens")
word_to_int={v:k for k,v in enumerate(words)} # k is number of word repetition and ,v is word
int_to_word={k:v for k,v in enumerate(words)}
print({k:v for k,v in word_to_int.items() if k in words[:10]})
print({k:v for k,v in int_to_word.items() if v in words[:10]})

the text contains 437098 words
there are 12778 unique tokens
{',': 0, '.': 1, 'the': 2, '"': 3, 'and': 4, 'to': 5, 'of': 6, 'he': 7, "'": 8, 'a': 9}
{0: ',', 1: '.', 2: 'the', 3: '"', 4: 'and', 5: 'to', 6: 'of', 7: 'he', 8: "'", 9: 'a'}


In [14]:
print(text[0:20])
wordidx=[word_to_int[w] for w in text]
print([word_to_int[w] for w in text[0:20]])  # The number of each word is the number of repetitions

['chapter', '1', 'happy', 'families', 'are', 'all', 'alike', ';', 'every', 'unhappy', 'family', 'is', 'unhappy', 'in', 'its', 'own', 'way', '.', 'everything', 'was']
[208, 2755, 280, 2981, 83, 31, 2419, 35, 202, 685, 362, 38, 685, 10, 236, 147, 166, 1, 149, 12]


## Create batches of training data

In [15]:
import torch

seq_len=100
xys=[]
for n in range(0, len(wordidx)-seq_len-1):  # 0 : 437098 -100 - 1
    x = wordidx[n:n+seq_len]                # x= 0:100   y = 1:101
    y = wordidx[n+1:n+seq_len+1]            # x= 1:101   y = 2:102
    xys.append((torch.tensor(x),(torch.tensor(y))))

In [16]:
 from torch.utils.data import DataLoader

torch.manual_seed(42)
batch_size=32
loader = DataLoader(xys, batch_size=batch_size, shuffle=True)

x,y=next(iter(loader))
print(x)
print(y)
print(x.shape,y.shape)

tensor([[  39,   31,    2,  ...,  688,  142,    7],
        [ 156, 5293,    0,  ...,   38,  330,    0],
        [   3,   97,    0,  ...,    0, 1774,   34],
        ...,
        [  16,  156,    9,  ...,  113,    5,  533],
        [   3,    4,   31,  ...,   98,    5,   98],
        [ 289,   19,   23,  ...,    9,  828,  550]])
tensor([[  31,    2, 2727,  ...,  142,    7,    0],
        [5293,    0,   16,  ...,  330,    0,    3],
        [  97,    0,    4,  ..., 1774,   34,    3],
        ...,
        [ 156,    9,  489,  ...,    5,  533,   27],
        [   4,   31,   25,  ...,    5,   98,    1],
        [  19,   23,    1,  ...,  828,  550,    1]])
torch.Size([32, 100]) torch.Size([32, 100])
